In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = pd.read_csv('https://raw.githubusercontent.com/passat05/Retail-data-analysis/272b60543e7f4a5197a0f463cf7076917df69293/Features%20data%20set.csv')
stores = pd.read_csv('https://raw.githubusercontent.com/passat05/Retail-data-analysis/272b60543e7f4a5197a0f463cf7076917df69293/stores%20data-set.csv')
sales = pd.read_csv('https://raw.githubusercontent.com/passat05/Retail-data-analysis/272b60543e7f4a5197a0f463cf7076917df69293/sales%20data-set.csv')

In [4]:
summary = pd.merge(sales, features.loc[:, features. columns != 'IsHoliday'],how = 'left', on = ['Store','Date'])

In [5]:
summary = pd.merge(summary, stores, how = 'left', on = ['Store'])

In [6]:
summary['Weekly_Sales'] = np.where(summary['Weekly_Sales'] < 0, 0, summary['Weekly_Sales'])

In [7]:
summary.fillna(0, inplace=True)

In [8]:
summary['Date'] = pd.to_datetime(summary['Date'], format='%d/%m/%Y')
summary['WeekOfYear'] = summary.Date.dt.isocalendar().week
summary['Year'] = summary.Date.dt.year

In [9]:
occasion = {
    '10-02': 'Super Bowl',
    '11-02': 'Super Bowl',
    '12-02': 'Super Bowl',
    # '13-02': 'Official Super Bowl'
    # '04-09': 'Official Labor Day',
    '07-09': 'Labor Day',
    '09-09': 'Labor Day',
    '10-09': 'Labor Day',
    '17-02': "Presidents' Day",
    '18-02': "Presidents' Day",
    '19-02': "Presidents' Day",
    '02-04': "Easter Day",
    '01-04': "Easter Day",
    '30-03': "Easter Day",
    '25-11': 'Thanksgiving',
    '26-11': 'Thanksgiving',
    '16-12': 'Before Christmas 1W',
    '17-12': 'Before Christmas 1W',
    '23-12': 'Christmas',
    '24-12': 'Christmas'
}

In [10]:
summary['Occasion'] = summary['Date'].dt.strftime("%d-%m").map(occasion)
summary['Occasion'].fillna('Normal', inplace = True)

In [11]:
summary['IsHoliday'] = summary['IsHoliday'].replace(True, 1).replace(False, 0)

In [12]:
summary.columns = summary.columns.str.lower()

In [13]:
summary.sort_values(by=['date'], inplace=True)

In [14]:
numerics_cols = [i for i in summary.select_dtypes(include='number').columns.to_list() if i not in 'weekly_sales']
categorical_cols = ['type','occasion']

In [15]:
one_hot_df = pd.get_dummies(summary[categorical_cols],prefix='_',prefix_sep='',dtype=float)
one_hot_cols = one_hot_df.columns
df = pd.merge(summary[numerics_cols + ['weekly_sales']], one_hot_df, left_index=True, right_index=True)

In [16]:
df.columns = df.columns.str.lower()

In [25]:
df_train = df[:int(.7*len(df))].reset_index(drop=True)
df_val = df[int(.7*len(df)):int(.85*len(df))].reset_index(drop=True)
df_test = df[int(.85*len(df)):].reset_index(drop=True)

In [26]:
y_train = np.log1p(df_train.weekly_sales)
y_val = np.log1p(df_val.weekly_sales)
y_test = np.log1p(df_test.weekly_sales)

In [27]:
del df_train['weekly_sales']
del df_val['weekly_sales']
del df_test['weekly_sales']

In [28]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler = MinMaxScaler().fit(df_train[numerics_cols])
for x in [df_train,df_val,df_test]:
    x[numerics_cols] = scaler.transform(x[numerics_cols])

In [30]:
X_train = df_train
X_val = df_val
X_test = df_test

In [31]:
from sklearn.ensemble import RandomForestRegressor

In [33]:
rf = RandomForestRegressor(n_estimators=50, random_state=1)
model = rf.fit(X_train, y_train)

In [34]:
with open("scaler_forecast_model.bin", "wb") as f_out:
    pickle.dump((scaler, model), f_out)

In [40]:
# import pickle
# with open("scaler_forecast_model.bin", "rb") as f_in:
#     scaler, _ = pickle.load(f_in)
#     # _, model = pickle.load(f_in)